In [3]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns

import os
import requests
import zipfile
import re

In [4]:
ranked_list = ['Relative Humidity', 'Solar Zenith Angle', 'GHI', 'Surface Albedo','Wind Direction','Temperature','Wind Speed','DNI','DHI','Dew Point','Pressure','Cloud Type']
feature_list = ranked_list[:11]
feature_list

['Relative Humidity',
 'Solar Zenith Angle',
 'GHI',
 'Surface Albedo',
 'Wind Direction',
 'Temperature',
 'Wind Speed',
 'DNI',
 'DHI',
 'Dew Point',
 'Pressure']

BMS station:
Latitude: 39.742o North
Longitude: 105.18o West

STAC station:
Latitude: 39.75685o North
Longitude: 104.62025o West

University of Oregon (SRML)
Latitude: 44.0467o North
Longitude: 123.0743o West

VTIF NREL Vehicle Testing and Integration Facility RSR
Latitude: 39.74211o North
Longitude: 105.17572o West

In [5]:
# acceptable location code: BMS, STAC, UOSMRL, VTIF
location='VTIF'

In [6]:
url = 'https://midcdmz.nrel.gov/apps/daily.pl?site='+location+'&live=1'
response = requests.get(url)
if response.status_code == 200:
    print ('Website accessed!')
else:
    print ('Error code 200. Rerun this code block or check website!')

Website accessed!


In [7]:
outfile = open("delete_after.txt", "w")
outfile.write(response.text)
outfile.close()

In [8]:
feature_dict=dict()
with open('delete_after.txt','r') as readfile:
    for line in readfile:
        result=re.search('<TD nowrap><DIV ALIGN=right>',line)
        if result is not None:
            for feature in ['Global','Direct','Diffuse','Wind Direction','Wind Speed','Zenith','Albedo','Relative Humidity','Air Temperature','Pressure']:
                if feature in line:
                    result=re.findall("[-+]?\d+\.\d+",line)
                    if result is not None:
                        if feature not in feature_dict:
                            feature_dict[feature]=float(result[0])

In [12]:
keys = list(feature_dict)
for key in keys:
    if key == "Global":
        feature_dict["GHI"] = feature_dict["Global"]
        del feature_dict["Global"]
    elif key == "Direct":
        feature_dict["DNI"] = feature_dict["Direct"]
        del feature_dict["Direct"]
    elif key == "Diffuse":
        feature_dict["DHI"] = feature_dict["Diffuse"]
        del feature_dict["Diffuse"]
    elif key == "Zenith":
        feature_dict["Solar Zenith Angle"] = feature_dict["Zenith"]
        del feature_dict["Zenith"]
    elif key == "Albedo":
        feature_dict["Surface Albedo"] = feature_dict["Albedo"]
        del feature_dict["Albedo"]
    elif key == 'Air Temperature':
        feature_dict["Temperature"] = feature_dict['Air Temperature']
        del feature_dict['Air Temperature']
    elif key == 'Pressure':
        feature_dict['Pressure'] = feature_dict['Pressure']*1.33322
    else:
        pass

In [13]:
feature_dict

{'Relative Humidity': 87.0,
 'Wind Speed': 0.0,
 'Wind Direction': 0.0,
 'Pressure': 1097.4533752,
 'GHI': 0.0,
 'DNI': 0.0,
 'DHI': 0.0,
 'Temperature': -0.24,
 'Solar Zenith Angle': 103.7}

In [91]:
! rm delete_after.txt